# Feature Computation: ΔS, ΔI, ΔC

This notebook demonstrates computation of the three deviation terms in the unified instability gate:

$$\Delta\Phi(t) = \alpha|\Delta S(t)| + \beta|\Delta I(t)| + \gamma|\Delta C(t)|$$

where:
- **ΔS**: Spectral/morphological deviation
- **ΔI**: Information/entropy deviation
- **ΔC**: Cross-modal coupling deviation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal, stats
from scipy.signal import hilbert, welch
from scipy.stats import entropy

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Spectral Deviation (ΔS)

Spectral deviation captures changes in frequency content relative to baseline.

For HRV, this includes:
- LF/HF ratio changes
- Power redistribution across bands
- Morphological drift (QRS/QT)

In [ ]:
def compute_spectral_features(signal_data, fs, nperseg=256):
    """
    Compute power spectral density and band powers.
    
    Parameters
    ----------
    signal_data : array_like
        Input time series
    fs : float
        Sampling frequency
    nperseg : int
        Length of each segment for Welch's method
    
    Returns
    -------
    features : dict
        Spectral features including band powers
    """
    # Compute PSD using Welch's method
    freqs, psd = welch(signal_data, fs=fs, nperseg=nperseg)
    
    # Define frequency bands
    delta = (0.5, 4)    # Delta band
    theta = (4, 8)      # Theta band
    alpha = (8, 13)     # Alpha band
    beta = (13, 30)     # Beta band
    gamma = (30, 50)    # Gamma band
    
    # For HRV
    vlf = (0.003, 0.04) # Very low frequency
    lf = (0.04, 0.15)   # Low frequency
    hf = (0.15, 0.4)    # High frequency
    
    def band_power(freqs, psd, band):
        """Calculate power in frequency band."""
        idx = np.logical_and(freqs >= band[0], freqs <= band[1])
        return np.trapz(psd[idx], freqs[idx])
    
    features = {
        'total_power': np.trapz(psd, freqs),
        'delta_power': band_power(freqs, psd, delta),
        'theta_power': band_power(freqs, psd, theta),
        'alpha_power': band_power(freqs, psd, alpha),
        'beta_power': band_power(freqs, psd, beta),
        'gamma_power': band_power(freqs, psd, gamma),
        'vlf_power': band_power(freqs, psd, vlf),
        'lf_power': band_power(freqs, psd, lf),
        'hf_power': band_power(freqs, psd, hf),
        'freqs': freqs,
        'psd': psd
    }
    
    # LF/HF ratio (autonomic balance marker)
    if features['hf_power'] > 0:
        features['lf_hf_ratio'] = features['lf_power'] / features['hf_power']
    else:
        features['lf_hf_ratio'] = np.nan
    
    return features


def compute_delta_S(current_features, baseline_features):
    """
    Compute spectral deviation from baseline.
    
    Parameters
    ----------
    current_features : dict
        Spectral features for current window
    baseline_features : dict
        Spectral features for baseline window
    
    Returns
    -------
    delta_s : float
        Normalized spectral deviation
    """
    # Compare total power
    power_dev = abs(
        current_features['total_power'] - baseline_features['total_power']
    ) / (baseline_features['total_power'] + 1e-10)
    
    # Compare LF/HF ratio (for HRV)
    if not np.isnan(current_features['lf_hf_ratio']) and \
       not np.isnan(baseline_features['lf_hf_ratio']):
        lf_hf_dev = abs(
            current_features['lf_hf_ratio'] - baseline_features['lf_hf_ratio']
        ) / (baseline_features['lf_hf_ratio'] + 1e-10)
    else:
        lf_hf_dev = 0
    
    # Combined spectral deviation
    delta_s = 0.5 * power_dev + 0.5 * lf_hf_dev
    
    return delta_s

## 2. Information Deviation (ΔI)

Information deviation measures changes in entropy and complexity.

Methods include:
- Sample entropy
- Approximate entropy
- Permutation entropy

In [ ]:
def sample_entropy(signal_data, m=2, r=0.2):
    """
    Compute sample entropy (complexity measure).
    
    Parameters
    ----------
    signal_data : array_like
        Input time series
    m : int
        Embedding dimension
    r : float
        Tolerance (as fraction of std)
    
    Returns
    -------
    sampen : float
        Sample entropy value
    """
    N = len(signal_data)
    r = r * np.std(signal_data)
    
    def _maxdist(x_i, x_j):
        return max([abs(ua - va) for ua, va in zip(x_i, x_j)])
    
    def _phi(m):
        x = [[signal_data[j] for j in range(i, i + m - 1 + 1)] 
             for i in range(N - m + 1)]
        C = [len([1 for j in range(len(x)) 
                  if i != j and _maxdist(x[i], x[j]) <= r]) 
             for i in range(len(x))]
        return sum(C)
    
    phi_m = _phi(m)
    phi_m_plus = _phi(m + 1)
    
    if phi_m == 0 or phi_m_plus == 0:
        return np.nan
    
    return -np.log(phi_m_plus / phi_m)


def permutation_entropy(signal_data, order=3, delay=1):
    """
    Compute permutation entropy.
    
    Parameters
    ----------
    signal_data : array_like
        Input time series
    order : int
        Permutation order
    delay : int
        Time delay
    
    Returns
    -------
    pe : float
        Permutation entropy
    """
    n = len(signal_data)
    permutations = {}
    
    for i in range(n - delay * (order - 1)):
        # Extract pattern
        pattern = signal_data[i:i + delay * order:delay]
        # Get permutation (rank order)
        perm = tuple(np.argsort(pattern))
        
        if perm in permutations:
            permutations[perm] += 1
        else:
            permutations[perm] = 1
    
    # Calculate entropy
    frequencies = np.array(list(permutations.values()))
    probabilities = frequencies / frequencies.sum()
    
    pe = entropy(probabilities)
    # Normalize
    pe = pe / np.log(np.math.factorial(order))
    
    return pe


def compute_information_features(signal_data):
    """
    Compute information-theoretic features.
    
    Returns
    -------
    features : dict
        Information features
    """
    features = {
        'sample_entropy': sample_entropy(signal_data),
        'permutation_entropy': permutation_entropy(signal_data),
    }
    
    return features


def compute_delta_I(current_features, baseline_features):
    """
    Compute information deviation from baseline.
    
    Parameters
    ----------
    current_features : dict
        Information features for current window
    baseline_features : dict
        Information features for baseline window
    
    Returns
    -------
    delta_i : float
        Normalized information deviation
    """
    # Sample entropy deviation
    if not np.isnan(current_features['sample_entropy']) and \
       not np.isnan(baseline_features['sample_entropy']):
        sampen_dev = abs(
            current_features['sample_entropy'] - baseline_features['sample_entropy']
        ) / (baseline_features['sample_entropy'] + 1e-10)
    else:
        sampen_dev = 0
    
    # Permutation entropy deviation
    permen_dev = abs(
        current_features['permutation_entropy'] - baseline_features['permutation_entropy']
    )
    
    # Combined information deviation
    delta_i = 0.5 * sampen_dev + 0.5 * permen_dev
    
    return delta_i

## 3. Coupling Deviation (ΔC)

Coupling deviation measures changes in cross-modal synchronization between EEG and ECG/HRV.

Methods:
- Phase synchronization index
- Coherence
- Mutual information

In [ ]:
def phase_synchronization_index(phi1, phi2):
    """
    Compute phase synchronization index between two signals.
    
    Parameters
    ----------
    phi1, phi2 : array_like
        Instantaneous phases of two signals
    
    Returns
    -------
    psi : float
        Phase synchronization index (0 to 1)
    """
    # Phase difference
    phase_diff = phi1 - phi2
    
    # Circular mean (order parameter)
    psi = np.abs(np.mean(np.exp(1j * phase_diff)))
    
    return psi


def compute_coherence(signal1, signal2, fs, nperseg=256):
    """
    Compute magnitude-squared coherence between two signals.
    
    Parameters
    ----------
    signal1, signal2 : array_like
        Input time series
    fs : float
        Sampling frequency
    nperseg : int
        Segment length
    
    Returns
    -------
    mean_coherence : float
        Mean coherence across frequencies
    """
    freqs, Cxy = signal.coherence(signal1, signal2, fs=fs, nperseg=nperseg)
    
    # Mean coherence
    mean_coherence = np.mean(Cxy)
    
    return mean_coherence


def compute_coupling_features(eeg_signal, ecg_signal, eeg_phase, ecg_phase, fs):
    """
    Compute cross-modal coupling features.
    
    Parameters
    ----------
    eeg_signal, ecg_signal : array_like
        Raw signals
    eeg_phase, ecg_phase : array_like
        Instantaneous phases
    fs : float
        Sampling frequency
    
    Returns
    -------
    features : dict
        Coupling features
    """
    features = {
        'phase_sync': phase_synchronization_index(eeg_phase, ecg_phase),
        'coherence': compute_coherence(eeg_signal, ecg_signal, fs)
    }
    
    return features


def compute_delta_C(current_features, baseline_features):
    """
    Compute coupling deviation from baseline.
    
    Parameters
    ----------
    current_features : dict
        Coupling features for current window
    baseline_features : dict
        Coupling features for baseline window
    
    Returns
    -------
    delta_c : float
        Normalized coupling deviation
    """
    # Phase synchronization deviation
    ps_dev = abs(
        current_features['phase_sync'] - baseline_features['phase_sync']
    )
    
    # Coherence deviation
    coh_dev = abs(
        current_features['coherence'] - baseline_features['coherence']
    )
    
    # Combined coupling deviation
    delta_c = 0.5 * ps_dev + 0.5 * coh_dev
    
    return delta_c

## 4. Demonstration with Synthetic Data

In [ ]:
# Generate synthetic EEG and HRV signals with regime change
np.random.seed(42)
fs = 250  # Hz
duration = 30  # seconds
t = np.linspace(0, duration, fs * duration)
transition_time = 15

# EEG: alpha to beta transition
alpha_amp = np.where(t < transition_time, 1.0, 0.3)
beta_amp = np.where(t < transition_time, 0.3, 1.0)
eeg = alpha_amp * np.sin(2 * np.pi * 10 * t) + \
      beta_amp * np.sin(2 * np.pi * 20 * t) + \
      0.2 * np.random.randn(len(t))

# HRV: LF/HF balance shift
lf_amp = np.where(t < transition_time, 5, 10)
hf_amp = np.where(t < transition_time, 10, 5)
hrv = 70 + lf_amp * np.sin(2 * np.pi * 0.1 * t) + \
      hf_amp * np.sin(2 * np.pi * 0.25 * t) + \
      2 * np.random.randn(len(t))

# Extract phases
eeg_phase = np.unwrap(np.angle(hilbert(eeg)))
hrv_phase = np.unwrap(np.angle(hilbert(hrv)))

In [ ]:
# Define baseline and evaluation windows
baseline_mask = t < 10
eval_mask = (t >= 15) & (t < 20)

# Compute features for baseline
baseline_spectral_eeg = compute_spectral_features(eeg[baseline_mask], fs)
baseline_spectral_hrv = compute_spectral_features(hrv[baseline_mask], fs)
baseline_info_eeg = compute_information_features(eeg[baseline_mask])
baseline_coupling = compute_coupling_features(
    eeg[baseline_mask], hrv[baseline_mask],
    eeg_phase[baseline_mask], hrv_phase[baseline_mask], fs
)

# Compute features for evaluation window
eval_spectral_eeg = compute_spectral_features(eeg[eval_mask], fs)
eval_spectral_hrv = compute_spectral_features(hrv[eval_mask], fs)
eval_info_eeg = compute_information_features(eeg[eval_mask])
eval_coupling = compute_coupling_features(
    eeg[eval_mask], hrv[eval_mask],
    eeg_phase[eval_mask], hrv_phase[eval_mask], fs
)

# Compute deviations
delta_s = compute_delta_S(eval_spectral_hrv, baseline_spectral_hrv)
delta_i = compute_delta_I(eval_info_eeg, baseline_info_eeg)
delta_c = compute_delta_C(eval_coupling, baseline_coupling)

print("=== Deviation Metrics ===")
print(f"ΔS (Spectral deviation): {delta_s:.4f}")
print(f"ΔI (Information deviation): {delta_i:.4f}")
print(f"ΔC (Coupling deviation): {delta_c:.4f}")

## 5. Unified Instability Functional

In [ ]:
def compute_instability_functional(delta_s, delta_i, delta_c, 
                                   alpha=0.4, beta=0.3, gamma=0.3):
    """
    Compute unified instability functional ΔΦ(t).
    
    Parameters
    ----------
    delta_s, delta_i, delta_c : float
        Individual deviation terms
    alpha, beta, gamma : float
        Weights (must sum to 1)
    
    Returns
    -------
    delta_phi : float
        Unified instability measure
    """
    assert abs(alpha + beta + gamma - 1.0) < 1e-6, "Weights must sum to 1"
    
    delta_phi = alpha * abs(delta_s) + beta * abs(delta_i) + gamma * abs(delta_c)
    
    return delta_phi


# Compute unified functional
delta_phi = compute_instability_functional(delta_s, delta_i, delta_c)

print(f"\nΔΦ(t) = {delta_phi:.4f}")
print(f"  = {0.4:.1f}×{delta_s:.4f} + {0.3:.1f}×{delta_i:.4f} + {0.3:.1f}×{delta_c:.4f}")

## 6. Visualization of Feature Space

In [ ]:
# ICE triangle visualization
fig, ax = plt.subplots(figsize=(8, 8))

# Triangle vertices (equilateral)
vertices = np.array([
    [0, 0],           # Information
    [1, 0],           # Energy
    [0.5, np.sqrt(3)/2]  # Coherence
])

triangle = plt.Polygon(vertices, fill=False, edgecolor='black', linewidth=2)
ax.add_patch(triangle)

# Convert (I, C, E) to barycentric coordinates
# Normalize deviations
total = delta_i + delta_c + delta_s
if total > 0:
    i_norm = delta_i / total
    c_norm = delta_c / total
    e_norm = delta_s / total
else:
    i_norm = c_norm = e_norm = 1/3

# Barycentric to Cartesian
point = i_norm * vertices[0] + e_norm * vertices[1] + c_norm * vertices[2]

# Plot baseline (center)
baseline_point = np.mean(vertices, axis=0)
ax.plot(*baseline_point, 'go', markersize=12, label='Baseline')

# Plot evaluation point
ax.plot(*point, 'ro', markersize=12, label='Evaluation')

# Arrow from baseline to evaluation
ax.arrow(baseline_point[0], baseline_point[1],
         point[0] - baseline_point[0], point[1] - baseline_point[1],
         head_width=0.03, head_length=0.03, fc='red', ec='red', alpha=0.6)

# Labels
ax.text(vertices[0][0]-0.1, vertices[0][1]-0.1, 'Information (I)',
        fontsize=12, ha='right')
ax.text(vertices[1][0]+0.1, vertices[1][1]-0.1, 'Energy (E)',
        fontsize=12, ha='left')
ax.text(vertices[2][0], vertices[2][1]+0.1, 'Coherence (C)',
        fontsize=12, ha='center')

ax.set_xlim(-0.2, 1.2)
ax.set_ylim(-0.2, 1.0)
ax.set_aspect('equal')
ax.axis('off')
ax.legend(loc='upper right')
ax.set_title('ICE Stability Triangle', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nICE coordinates (normalized):")
print(f"  Information: {i_norm:.3f}")
print(f"  Coherence:   {c_norm:.3f}")
print(f"  Energy:      {e_norm:.3f}")

## Summary

This notebook demonstrated:

1. **ΔS computation**: Spectral/morphological deviation (LF/HF ratio, band powers)
2. **ΔI computation**: Information-theoretic deviation (entropy measures)
3. **ΔC computation**: Cross-modal coupling deviation (phase sync, coherence)
4. **ΔΦ functional**: Weighted combination with transparent parameters
5. **ICE visualization**: Interpretive mapping in information-coherence-energy space

All features are deterministic, traceable, and admit direct ablation testing.

**Next steps**: See `03_eeg_validation.ipynb` for real EEG dataset validation.